In [ ]:
class Encoder(nn.Module):
    def __init__(self, **kwargs):
        super(Encoder, self).__init__()

class Decoder(nn.Module):
    def __init__(self, **kwargs):
        super(Decoder, self).__init__()
    def init_state(self, enc_out):  
        """Initialize the decoder's state."""
        raise NotImplementedError
    def forward(self, inputs, state):
        raise NotImplementedError
    
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, **kwargs):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.init_weights()
        
    def init_weights(self):
        """Initialize the weights."""
        raise NotImplementedError
    
    def forward(self, enc_x, dec_x,**kwargs):
        enc_outputs=self.encoder(enc_x,**kwargs)
        dec_state=self.decoder.init_state(enc_outputs,**kwargs)
        return self.decoder(dec_x,dec_state)
    


<img src="../images/seq2seq模型.jpg" width="60%">

In [ ]:
class Seq2SeqEncoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers=1, dropout=0.0,**kwargs):
        super(Seq2SeqEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(embed_size, num_hiddens, num_layers, dropout=dropout, batch_first=True)
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers

    def begin_state(self, batch_size=1, device=torch.device('cpu'), dtype=torch.float32):
        return (torch.zeros((self.num_layers, batch_size, self.num_hiddens), device=device, dtype=dtype),
                torch.zeros((self.num_layers, batch_size, self.num_hiddens), device=device, dtype=dtype))

    def forward(self, inputs, state=None):
        X = self.embedding(inputs)
        X=X.transpose(0,1)#batch_size,seq_len,embed_size
        out,state=self.rnn(X)
        return out, state



class Seq2SeqDecoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers=1, dropout=0.0,**kwargs):
        super(Seq2SeqDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(embed_size, num_hiddens, num_layers, dropout=dropout, batch_first=True)
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, batch_size=1, device=torch.device('cpu'), dtype=torch.float32):
        return enc_outputs[1]

    def forward(self, inputs,state=None):
        X = self.embedding(inputs)
        X=X.transpose(0,1)#batch_size,seq_len,embed_size
        out,state=self.rnn(X,state)
        out=self.dense(out).transpose(0,1)
        return out, state
    

## 注意力机制框架
<img src="../images/注意力机制框架.jpg" width="60%">